In [2]:
from datasets import load_dataset 

In [3]:
ds = load_dataset("laion/biorXiv_metadata")
print(ds)

DatasetDict({
    train: Dataset({
        features: ['doi', 'title', 'authors', 'author_corresponding', 'author_corresponding_institution', 'date', 'version', 'type', 'license', 'category', 'jatsxml', 'abstract', 'published', 'server'],
        num_rows: 353648
    })
})


In [97]:
df = ds["train"].to_pandas()

In [98]:
df = df.drop_duplicates("doi").copy()

In [99]:
def tokenizer_encode(strings: list[str], alphabet: str):
	index_map = {
		"<s>": 0,
		"<e>": 1,
	}
	for i in range(len(alphabet)):
		index_map[alphabet[i]] = i+2
	
	res = []
	for s in strings:
		sub = [0] # <s>
		for c in s:
			sub.append(index_map[c])
		sub.append(1) # <e>
		res.append(sub)
	return res

def tokenizer_decode(idxs: list[list[int]], alphabet: str):
	reverse_index_map = ["<s>", "<e>"]
	reverse_index_map.extend(alphabet)


	res = []
	for sub in idxs:
		string = ""
		for i in sub:
			string += reverse_index_map[i]
		res.append(string)
	return res

alphabet=[' ','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','?','!','.',',',':',';','&','%','1','2','3','4','5','6','7','8','9','0', '-']

In [100]:
import re
def parse_title(x: str, alphabet: set[str]):
	x = re.sub(r'\s', ' ', x)
	x = re.sub(r'–', '-', x)
	x = x.lower()
	res = ""
	for c in x:
		if c in alphabet:
			res += c
	return res

alphabet=[' ','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','?','.',',',':','&','%','1','2','3','4','5','6','7','8','9','0']
set_alpha = set(alphabet)


df["parsed_title"] = df["title"].apply(lambda x: parse_title(x, set_alpha))

In [104]:
subdf = df[["doi", "parsed_title"]]
subdf

,doi,parsed_title
0,10.1101/001891,population genomics of saccharomyces cerevisia...
1,10.1101/001867,estimating seed bank accumulation and dynamics...
2,10.1101/001875,how and where to look for trnas in metazoan mi...
4,10.1101/001883,tracking global changes induced in the cd4 t c...
5,10.1101/001909,the shrinking human protein coding complement:...
...,...,...
353643,10.1101/2024.11.08.622749,identification of covalent cyclic peptide inhi...
353644,10.1101/2024.11.08.622755,carotenoid pigments enhance rhodopsinmediated ...
353645,10.1101/2024.11.08.622053,a bacterial cell wall repair and modification ...
353646,10.1101/2024.11.09.622756,characterization of a gpsbassociated regulator...
